<a href="https://colab.research.google.com/github/uldry/data/blob/main/Spark_NYC_YellowCab_2009_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spark_NYC_YellowCab-2009-2024

Data Download, Transformation and  Exploration/Cleaning of NYC Yellow Cab dataset from 2009 to 2024 using Google Colab. about ~1.8 billion rows. As the resuling dataset couldn't be saved anywhere due to a stalling issue and colab crashing, the work stopped there.

Setting up pySpark and starting the session

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3-scala2.13.tgz
!tar xf spark-3.5.3-bin-hadoop3-scala2.13.tgz
!pip install -q findspark
# remove?
#!pip install pyspark_dist_explore

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,199 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,584 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.3-bin-hadoop3-scala2.13"
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName("Test Setup")\
        .getOrCreate()

Imports

In [ ]:
import pyspark
print(pyspark.__version__)

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, lit, array, struct, col, lower, create_map
from pyspark.sql.functions import to_timestamp_ntz, unix_timestamp
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType, ShortType, IntegerType, StructType, StructField, ArrayType
from pyspark.sql.types import StringType,LongType, IntegerType, DoubleType
from pyspark.ml.feature import Bucketizer


import numpy as np
import pandas as pd
import math
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns

import shapefile
import geopandas as gpd
from shapely.geometry import Point

from google.colab import drive
from google.colab import userdata



3.5.3


### Downloading the data from the repository

Adjust to download the necessary data

In [ ]:
!wget -nc https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_20{09..18}-{01..12}.parquet

File ‘yellow_tripdata_2009-01.parquet’ already there; not retrieving.

File ‘yellow_tripdata_2009-02.parquet’ already there; not retrieving.

File ‘yellow_tripdata_2009-03.parquet’ already there; not retrieving.

File ‘yellow_tripdata_2009-04.parquet’ already there; not retrieving.

File ‘yellow_tripdata_2009-05.parquet’ already there; not retrieving.

File ‘yellow_tripdata_2009-06.parquet’ already there; not retrieving.

File ‘yellow_tripdata_2009-07.parquet’ already there; not retrieving.

File ‘yellow_tripdata_2009-08.parquet’ already there; not retrieving.

File ‘yellow_tripdata_2009-09.parquet’ already there; not retrieving.

File ‘yellow_tripdata_2009-10.parquet’ already there; not retrieving.

File ‘yellow_tripdata_2009-11.parquet’ already there; not retrieving.

File ‘yellow_tripdata_2009-12.parquet’ already there; not retrieving.

File ‘yellow_tripdata_2010-01.parquet’ already there; not retrieving.

File ‘yellow_tripdata_2010-02.parquet’ already there; not retrieving.

File ‘

**Wait some time here to not exceed the download limit**

In [ ]:
!wget -nc https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_20{19..24}-{01..12}.parquet

File ‘yellow_tripdata_2019-01.parquet’ already there; not retrieving.

File ‘yellow_tripdata_2019-02.parquet’ already there; not retrieving.

File ‘yellow_tripdata_2019-03.parquet’ already there; not retrieving.

File ‘yellow_tripdata_2019-04.parquet’ already there; not retrieving.

File ‘yellow_tripdata_2019-05.parquet’ already there; not retrieving.

File ‘yellow_tripdata_2019-06.parquet’ already there; not retrieving.

File ‘yellow_tripdata_2019-07.parquet’ already there; not retrieving.

File ‘yellow_tripdata_2019-08.parquet’ already there; not retrieving.

File ‘yellow_tripdata_2019-09.parquet’ already there; not retrieving.

File ‘yellow_tripdata_2019-10.parquet’ already there; not retrieving.

File ‘yellow_tripdata_2019-11.parquet’ already there; not retrieving.

File ‘yellow_tripdata_2019-12.parquet’ already there; not retrieving.

File ‘yellow_tripdata_2020-01.parquet’ already there; not retrieving.

File ‘yellow_tripdata_2020-02.parquet’ already there; not retrieving.

File ‘

Downloading location data

In [ ]:
!wget -nc 'https://d37ci6vzurychx.cloudfront.net/misc/taxi_zones.zip'
!wget -nc 'https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv'
!unzip -n taxi_zones.zip

--2025-01-12 18:21:55--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zones.zip
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 52.84.160.213, 52.84.160.84, 52.84.160.73, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|52.84.160.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1025147 (1001K) [application/zip]
Saving to: ‘taxi_zones.zip’

taxi_zones.zip      100%[===================>]   1001K  --.-KB/s    in 0.05s   

2025-01-12 18:21:55 (18.7 MB/s) - ‘taxi_zones.zip’ saved [1025147/1025147]

--2025-01-12 18:21:55--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 52.84.160.213, 52.84.160.84, 52.84.160.73, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|52.84.160.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘t

### Loading the data
There are 3 different schema versions, 2009, 2010 and 2011 onwards.

The oldest schemas will be transformed into the lastest version, some columns may be dropped

#### Recent schema, 2011 onwards

In [ ]:
parquet_files=[]
for i in range(11,25):
  parquet_files.extend(glob.glob( f'*20{str(i)}*.parquet'))
parquet_files.sort()  # Ensure sequential order

#parquet_files =['yellow_tripdata_2011-01.parquet']

In [ ]:
df3 = None
for file_path in parquet_files:
    if df3 is None:
        df3 = spark.read.parquet(file_path)
    else:
        #df = spark.read.parquet(file_path)
        df3 = df3.union(spark.read.parquet(file_path))

In [ ]:
df3.dtypes

[('VendorID', 'bigint'),
 ('tpep_pickup_datetime', 'timestamp_ntz'),
 ('tpep_dropoff_datetime', 'timestamp_ntz'),
 ('passenger_count', 'double'),
 ('trip_distance', 'double'),
 ('RatecodeID', 'double'),
 ('store_and_fwd_flag', 'string'),
 ('PULocationID', 'bigint'),
 ('DOLocationID', 'bigint'),
 ('payment_type', 'bigint'),
 ('fare_amount', 'double'),
 ('extra', 'double'),
 ('mta_tax', 'double'),
 ('tip_amount', 'double'),
 ('tolls_amount', 'double'),
 ('improvement_surcharge', 'double'),
 ('total_amount', 'double'),
 ('congestion_surcharge', 'double'),
 ('airport_fee', 'double')]

Dropping column that are not interesting

In [ ]:
df3 = df3.drop('airport_fee').drop('store_and_fwd_flag').drop('airport_fee').drop('RatecodeID').drop('improvement_surcharge').drop('congestion_surcharge')\
      .drop('extra').drop('mta_tax').drop('tolls_amount').drop('total_amount').drop('VendorID')

Casting to Short Int whereever possible to save memory

In [ ]:
df3 = df3.withColumn('PULocationID', col('PULocationID').cast(ShortType()))\
.withColumn('DOLocationID', col('DOLocationID').cast(ShortType()))\
.withColumn('payment_type', col('payment_type').cast(ShortType()))\
.withColumn('passenger_count', col('passenger_count').cast(ShortType()))

In [ ]:
df3.count()

1431026083

#### 2009 Schema

In [ ]:
parquet_files=[]

parquet_files.extend(glob.glob( f'*2009*.parquet'))
parquet_files.sort()  # Ensure sequential order

df1 = None
for file_path in parquet_files:
    if df1 is None:
        df1 = spark.read.parquet(file_path)
    else:
       df1 = df1.union(spark.read.parquet(file_path))

In [ ]:
df1.dtypes

[('vendor_name', 'string'),
 ('Trip_Pickup_DateTime', 'string'),
 ('Trip_Dropoff_DateTime', 'string'),
 ('Passenger_Count', 'bigint'),
 ('Trip_Distance', 'double'),
 ('Start_Lon', 'double'),
 ('Start_Lat', 'double'),
 ('Rate_Code', 'double'),
 ('store_and_forward', 'double'),
 ('End_Lon', 'double'),
 ('End_Lat', 'double'),
 ('Payment_Type', 'string'),
 ('Fare_Amt', 'double'),
 ('surcharge', 'double'),
 ('mta_tax', 'double'),
 ('Tip_Amt', 'double'),
 ('Tolls_Amt', 'double'),
 ('Total_Amt', 'double')]

In [ ]:
df1.count()

170896055

In [ ]:
df1.show(1)

+-----------+--------------------+---------------------+---------------+-------------+----------+---------+---------+-----------------+----------+---------+------------+--------+---------+-------+-------+---------+---------+
|vendor_name|Trip_Pickup_DateTime|Trip_Dropoff_DateTime|Passenger_Count|Trip_Distance| Start_Lon|Start_Lat|Rate_Code|store_and_forward|   End_Lon|  End_Lat|Payment_Type|Fare_Amt|surcharge|mta_tax|Tip_Amt|Tolls_Amt|Total_Amt|
+-----------+--------------------+---------------------+---------------+-------------+----------+---------+---------+-----------------+----------+---------+------------+--------+---------+-------+-------+---------+---------+
|        VTS| 2009-01-04 02:52:00|  2009-01-04 03:02:00|              1|         2.63|-73.991957|40.721567|     NULL|             NULL|-73.993803|40.695922|        CASH|     8.9|      0.5|   NULL|    0.0|      0.0|      9.4|
+-----------+--------------------+---------------------+---------------+-------------+----------+---

Vendor Name in the older data set is now irrelevant, but the type must match the new schema

In [ ]:
df1 = df1.drop('store_and_forward').drop('rate_code').drop('surcharge').drop('mta_tax').drop('tolls_Amt').drop('Total_Amt').drop('Vendor_Name')

A few columns are renamed

In [ ]:
df1 = df1.withColumnRenamed('Fare_Amt', 'fare_amount').withColumnRenamed('Tip_Amt', 'tip_amount')

In [ ]:
df1.show(1)

+-----------+--------------------+---------------------+---------------+-------------+----------+---------+----------+---------+------------+-----------+----------+
|vendor_name|Trip_Pickup_DateTime|Trip_Dropoff_DateTime|Passenger_Count|Trip_Distance| Start_Lon|Start_Lat|   End_Lon|  End_Lat|Payment_Type|fare_amount|tip_amount|
+-----------+--------------------+---------------------+---------------+-------------+----------+---------+----------+---------+------------+-----------+----------+
|        VTS| 2009-01-04 02:52:00|  2009-01-04 03:02:00|              1|         2.63|-73.991957|40.721567|-73.993803|40.695922|        CASH|        8.9|       0.0|
+-----------+--------------------+---------------------+---------------+-------------+----------+---------+----------+---------+------------+-----------+----------+
only showing top 1 row



`Payment_Type` has to be converted according to the data dictionary




In [ ]:
df1 = df1.withColumn('Payment_Type', lower(col('Payment_Type')))

In [ ]:
#df1.select('Payment_Type').distinct().collect()

In [ ]:
df1 = df1.withColumn(
    "payment_Type",
      F.when(col("Payment_Type") == "credit", 1)
     .when(col("Payment_Type") == "cash", 2)
     .when(col("Payment_Type") == "no charge", 3)
     .when(col("Payment_Type") == "dispute", 4)
     .when(col("Payment_Type") == "unknown", 5)
     .when(col("Payment_Type") == "voided trip", 6)
     .otherwise(None)  # Handle any unexpected values
)

In [ ]:
df1 = df1.withColumn('payment_type', col('payment_type').cast(ShortType()))\
.withColumn('passenger_count', col('passenger_count').cast(ShortType()))

The datetime are converted to the `timestamp_ntz` type

In [ ]:
df1 = df1.withColumn('tpep_pickup_datetime', to_timestamp_ntz(df1['Trip_Pickup_DateTime'])).drop('Trip_Pickup_DateTime')
df1 = df1.withColumn('tpep_dropoff_datetime', to_timestamp_ntz(df1['Trip_Dropoff_DateTime'])).drop('Trip_Dropoff_DateTime')

In [ ]:
df1.show(1)

+-----------+---------------+-------------+----------+---------+----------+---------+------------+-----------+----------+--------------------+---------------------+
|vendor_name|passenger_count|Trip_Distance| Start_Lon|Start_Lat|   End_Lon|  End_Lat|payment_type|fare_amount|tip_amount|tpep_pickup_datetime|tpep_dropoff_datetime|
+-----------+---------------+-------------+----------+---------+----------+---------+------------+-----------+----------+--------------------+---------------------+
|        VTS|              1|         2.63|-73.991957|40.721567|-73.993803|40.695922|           2|        8.9|       0.0| 2009-01-04 02:52:00|  2009-01-04 03:02:00|
+-----------+---------------+-------------+----------+---------+----------+---------+------------+-----------+----------+--------------------+---------------------+
only showing top 1 row



Converting Lat/Long into the locationID


The problem here is to get a borough and location ID from a lat long.
The X-Y format is also not WGS84...

In [ ]:
from math import isnan
# Create a point from lat/long
def get_locID(lat, long):
  #lat = 40.695922; long= -73.993803
  point = gpd.GeoDataFrame(geometry=[Point(long, lat)],crs="EPSG:4326")
  point = point.to_crs(epsg=2263)
  # Load borough shapefile
  boroughs = gpd.read_file('taxi_zones.shp')

  # Perform spatial join
  result = gpd.sjoin(point, boroughs, how="left", predicate='intersects')
  locid = result.loc[0, 'LocationID']
  if math.isnan(locid):
    return(265)
  else:
    return(int(locid))

  return()

In [ ]:
 # print(get_locID(40.695922, -73.993803))

# Registering the UDF with Spark
get_locID_udf = udf(lambda lat, long: get_locID(lat, long), ShortType())

In [ ]:
df1 = df1.withColumn("PULocationID", get_locID_udf(df1.Start_Lat, df1.Start_Lon))\
  .withColumn("DOLocationID", get_locID_udf(df1.End_Lat, df1.End_Lon))\
  .drop('Start_Lon', 'Start_Lat', 'End_Lon', 'End_Lat')

In [ ]:
df1.show(2)

+-----------+---------------+-------------+------------+-----------+----------+--------------------+---------------------+------------+------------+
|vendor_name|passenger_count|Trip_Distance|payment_type|fare_amount|tip_amount|tpep_pickup_datetime|tpep_dropoff_datetime|PULocationID|DOLocationID|
+-----------+---------------+-------------+------------+-----------+----------+--------------------+---------------------+------------+------------+
|        VTS|              1|         2.63|           2|        8.9|       0.0| 2009-01-04 02:52:00|  2009-01-04 03:02:00|         148|          33|
|        VTS|              3|         4.55|           1|       12.1|       2.0| 2009-01-04 03:31:00|  2009-01-04 03:38:00|         107|         141|
+-----------+---------------+-------------+------------+-----------+----------+--------------------+---------------------+------------+------------+
only showing top 2 rows



In [ ]:
#df1.select('payment_type').distinct().collect()

In [ ]:
df1.dtypes

In [ ]:
df1.count()

#### 2010 Schema

In [ ]:
parquet_files=[]

parquet_files.extend(glob.glob( f'*2010*.parquet'))
parquet_files.sort()  # Ensure sequential order

df2 = None
for file_path in parquet_files:
    if df2 is None:
        df2 = spark.read.parquet(file_path)
    else:
      dt  = spark.read.parquet(file_path)
      # removing some extra columns that are present in some files
      columns_to_drop = [col for col in dt.columns if col.startswith("__")]
      dt = dt.drop(*columns_to_drop)

      df2 = df2.union(dt)

In [ ]:
df2.dtypes

[('vendor_id', 'string'),
 ('pickup_datetime', 'string'),
 ('dropoff_datetime', 'string'),
 ('passenger_count', 'bigint'),
 ('trip_distance', 'double'),
 ('pickup_longitude', 'double'),
 ('pickup_latitude', 'double'),
 ('rate_code', 'string'),
 ('store_and_fwd_flag', 'string'),
 ('dropoff_longitude', 'double'),
 ('dropoff_latitude', 'double'),
 ('payment_type', 'string'),
 ('fare_amount', 'double'),
 ('surcharge', 'double'),
 ('mta_tax', 'double'),
 ('tip_amount', 'double'),
 ('tolls_amount', 'double'),
 ('total_amount', 'double')]

In [ ]:
df2.show()

+---------+-------------------+-------------------+---------------+------------------+------------------+---------------+---------+------------------+------------------+----------------+------------+-----------+---------+-------+----------+------------+-----------------+
|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|     trip_distance|  pickup_longitude|pickup_latitude|rate_code|store_and_fwd_flag| dropoff_longitude|dropoff_latitude|payment_type|fare_amount|surcharge|mta_tax|tip_amount|tolls_amount|     total_amount|
+---------+-------------------+-------------------+---------------+------------------+------------------+---------------+---------+------------------+------------------+----------------+------------+-----------+---------+-------+----------+------------+-----------------+
|      VTS|2010-01-26 07:41:00|2010-01-26 07:45:00|              1|              0.75|        -73.956778|       40.76775|        1|              NULL|        -73.965957|       40.76523

In [ ]:
df2 = df2.drop('store_and_fwd_flag').drop('rate_code').drop('surcharge').drop('mta_tax').drop('tolls_amount').drop('total_amount').drop('vendor_id')

In [ ]:
df2 = df2.withColumn('Payment_Type', lower(col('Payment_Type')))

In [ ]:
df2.select('payment_type').distinct().collect()

[Row(payment_type='cas'),
 Row(payment_type='cre'),
 Row(payment_type='dis'),
 Row(payment_type='no '),
 Row(payment_type='crd'),
 Row(payment_type='csh'),
 Row(payment_type='na '),
 Row(payment_type='noc')]

In [ ]:
df2 = df2.withColumn(
    "Payment_Type",
      F.when(col("Payment_Type") == "cre", 1)
     .when(col("Payment_Type") == "cas", 2)
     .when(col("Payment_Type") == "no", 3)
     .when(col("Payment_Type") == "dis", 4)
     .when(col("Payment_Type") == "unknown", 5)
     .when(col("Payment_Type") == "voided trip", 6)
     .otherwise(None)  # Handle any unexpected values
)

In [ ]:
df2 = df2.withColumn('payment_type', col('payment_type').cast(ShortType()))\
.withColumn('passenger_count', col('passenger_count').cast(ShortType()))

In [ ]:
df2 = df2.withColumn('tpep_pickup_datetime', to_timestamp_ntz(df2['pickup_datetime'])).drop('pickup_datetime')\
    .withColumn('tpep_dropoff_datetime', to_timestamp_ntz(df2['dropoff_DateTime'])).drop('dropoff_DateTime')

In [ ]:
df2 = df2.withColumn("PULocationID", get_locID_udf(df2.pickup_latitude, df2.pickup_longitude))\
  .withColumn("DOLocationID", get_locID_udf(df2.dropoff_latitude, df2.dropoff_longitude))\
  .drop('pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude')

In [ ]:
df2.show(1)

+---------------+-------------+------------+-----------+----------+--------------------+---------------------+------------+------------+
|passenger_count|trip_distance|payment_type|fare_amount|tip_amount|tpep_pickup_datetime|tpep_dropoff_datetime|PULocationID|DOLocationID|
+---------------+-------------+------------+-----------+----------+--------------------+---------------------+------------+------------+
|              1|         0.75|           2|        4.5|       0.0| 2010-01-26 07:41:00|  2010-01-26 07:45:00|         141|         237|
+---------------+-------------+------------+-----------+----------+--------------------+---------------------+------------+------------+
only showing top 1 row



### Union of dataframes

In [ ]:
a = df1.count()
b = df2.count()
c = df3.count()

In [ ]:
print(a,b,c)
print(a+b+c)

170896055 169001162 1431026083
1770923300


In [ ]:
df=[]
df = df1.unionByName(df2)
df = df.unionByName(df3)

In [ ]:
#del df1, df2, df3

In [ ]:
df.count()

1770923300

# Data Exploration

Let's explore and clean

In [ ]:
df.dtypes

[('passenger_count', 'smallint'),
 ('Trip_Distance', 'double'),
 ('payment_type', 'smallint'),
 ('fare_amount', 'double'),
 ('tip_amount', 'double'),
 ('tpep_pickup_datetime', 'timestamp_ntz'),
 ('tpep_dropoff_datetime', 'timestamp_ntz'),
 ('PULocationID', 'smallint'),
 ('DOLocationID', 'smallint')]

#### **Passenger count**

In [ ]:
df.select('passenger_count').describe().toPandas()

,summary,passenger_count
0,count,1762062885
1,mean,1.640041206588379
2,stddev,1.2849821073503065
3,min,0
4,max,255


In [ ]:
dg = df.groupBy("passenger_count").count()..orderBy("passenger_count")show()

Looking at the distribution, it was decided to drop all trips with more than 9 passenger

In [ ]:
df = df.where(df.passenger_count < 9)

#### **Trip distance**

In [ ]:
df.select("Trip_Distance").describe().toPandas()

,summary,Trip_Distance
0,count,1762059623
1,mean,5.582044916752768
2,stddev,6031.371714021165
3,min,-4.08401244E7
4,max,1.346190631E8


In [ ]:
df = df.where(df.Trip_Distance > 0)

In [ ]:
from pyspark.ml.feature import Bucketizer

In order to have an idea of the trip distance distribution the a few bins are created

In [ ]:
bin_labels = spark.createDataFrame([
    (0, "0-10"),
    (1, "10-20"),
    (2, "20-50"),
    (3, "50-100"),
    (4, "100-200"),
    (5, "200-500"),
    (6, "500+"),
], ["bins", "range"])

splits=[0, 10, 20, 50, 100, 200, 500,float('Inf')]

bucketizer = Bucketizer(splits=splits,
                        inputCol='Trip_Distance',
                        outputCol='bins')
df_bins = bucketizer.setHandleInvalid('keep').transform(df)

df_binned = df_bins.groupBy('bins').count()

result = (df_binned
    .join(bin_labels, "bins")
    .select("range", "count","bins").orderBy("bins")
)

result.show()

+-------+----------+----+
|  range|     count|bins|
+-------+----------+----+
|   0-10|1656700496| 0.0|
|  10-20|  83415828| 1.0|
|  20-50|   8961411| 2.0|
| 50-100|     74439| 3.0|
|100-200|      7058| 4.0|
|200-500|      1800| 5.0|
|   500+|      3117| 6.0|
+-------+----------+----+



Trips over 500 miles: The distances, fares, trip durations don't make sense.

In [ ]:
df.filter(df.Trip_Distance >500).show()

+---------------+-------------+------------+-----------+----------+--------------------+---------------------+------------+------------+
|passenger_count|Trip_Distance|payment_type|fare_amount|tip_amount|tpep_pickup_datetime|tpep_dropoff_datetime|PULocationID|DOLocationID|
+---------------+-------------+------------+-----------+----------+--------------------+---------------------+------------+------------+
|              1|     335550.0|        NULL|      20.18|       0.0| 2010-02-15 12:06:00|  2010-02-15 12:32:00|          68|         166|
|              1|       657.57|        NULL|        9.7|      2.28| 2010-02-11 11:27:00|  2010-02-11 11:43:00|         186|          79|
|              1|     260050.0|           2|       12.5|       0.0| 2010-02-03 01:16:47|  2010-02-03 01:31:54|         107|           7|
|              1|        535.7|        NULL|        0.0|       0.0| 2010-02-05 00:58:41|  2010-02-05 02:13:46|         246|          41|
|              1|        500.9|          

filtering the trips of more than 100 miles using the fare per mile helps. ( the first mile costs 4.5 USD, the subsequent ones cost 2.5 USD)
However, for some, the irrealistic trip duration in relation to distance traveled has to be dealt with.

In [ ]:
df.where((df.Trip_Distance > 100) & (df.fare_amount>=(df.Trip_Distance*2.5))).show()

+---------------+-------------+------------+-----------+----------+--------------------+---------------------+------------+------------+
|passenger_count|Trip_Distance|payment_type|fare_amount|tip_amount|tpep_pickup_datetime|tpep_dropoff_datetime|PULocationID|DOLocationID|
+---------------+-------------+------------+-----------+----------+--------------------+---------------------+------------+------------+
|              1|        112.6|           2|      454.5|       0.0| 2010-02-11 08:04:30|  2010-02-11 10:08:40|          10|         265|
|              1|       139.05|        NULL|      450.0|       0.0| 2010-03-13 22:49:00|  2010-03-14 01:26:00|         138|         265|
|              1|       134.96|        NULL|      495.7|       0.0| 2010-03-20 05:14:00|  2010-03-20 07:34:00|         132|         265|
|              1|        500.0|           2|    1400.07|       0.0| 2010-03-02 16:57:08|  2010-03-02 17:08:44|         230|         246|
|              1|        663.8|        NU

In [ ]:
df.where((df.Trip_Distance > 100) & (df.fare_amount>=(df.Trip_Distance*2.5))) \
.withColumn('mph', col("Trip_Distance") / (\
        (unix_timestamp(col("tpep_dropoff_datetime")) - unix_timestamp(col("tpep_pickup_datetime"))) / 3600.0\
    ).cast(DoubleType())).show()

+---------------+-------------+------------+-----------+----------+--------------------+---------------------+------------+------------+------------------+
|passenger_count|Trip_Distance|payment_type|fare_amount|tip_amount|tpep_pickup_datetime|tpep_dropoff_datetime|PULocationID|DOLocationID|               mph|
+---------------+-------------+------------+-----------+----------+--------------------+---------------------+------------+------------+------------------+
|              1|        112.6|           2|      454.5|       0.0| 2010-02-11 08:04:30|  2010-02-11 10:08:40|          10|         265| 54.41073825503355|
|              1|       139.05|        NULL|      450.0|       0.0| 2010-03-13 22:49:00|  2010-03-14 01:26:00|         138|         265|53.140127388535035|
|              1|       134.96|        NULL|      495.7|       0.0| 2010-03-20 05:14:00|  2010-03-20 07:34:00|         132|         265|57.839999999999996|
|              1|        500.0|           2|    1400.07|       0

In [ ]:
d100 = df.where(
    (df.Trip_Distance > 100) &
    (df.fare_amount >= (df.Trip_Distance * 2.5)) &
    # Speed = distance / time (in hours)
    (df.Trip_Distance /
     (unix_timestamp(df.tpep_dropoff_datetime) - unix_timestamp(df.tpep_pickup_datetime)) * 3600.0 <= 60.0)
)

In [ ]:
d100.show()

+---------------+-------------+------------+-----------+----------+--------------------+---------------------+------------+------------+
|passenger_count|Trip_Distance|payment_type|fare_amount|tip_amount|tpep_pickup_datetime|tpep_dropoff_datetime|PULocationID|DOLocationID|
+---------------+-------------+------------+-----------+----------+--------------------+---------------------+------------+------------+
|              1|        112.6|           2|      454.5|       0.0| 2010-02-11 08:04:30|  2010-02-11 10:08:40|          10|         265|
|              1|       139.05|        NULL|      450.0|       0.0| 2010-03-13 22:49:00|  2010-03-14 01:26:00|         138|         265|
|              1|       134.96|        NULL|      495.7|       0.0| 2010-03-20 05:14:00|  2010-03-20 07:34:00|         132|         265|
|              1|       113.05|        NULL|      350.0|      40.0| 2010-03-31 19:18:00|  2010-03-31 21:17:00|         138|         265|
|              1|       106.42|        NU

Here we're effectively dropping all the trips over 100 mi. where the speed and total fare are irrealistic.
Instead of dropping, the trip distance could have been set to Null or to the median value of trip using the pickup and dropoff zone IDs

Also, the trips of less than 100 mi. are not verified, hoping that won't have a large impact overall.

Drop all the trips over 100 mi.

In [ ]:
df = df.where(df.Trip_Distance <= 100)

Add the filtered trips that are over 100 mi.

In [ ]:
df =  df.unionByName(d100)

In [ ]:
df.count()

1749154119

#### **payment_type**

In [ ]:
hist_payment = df.groupBy("payment_type").count()
hist_payment.orderBy("payment_type").show()

+------------+---------+
|payment_type|    count|
+------------+---------+
|        NULL| 80855892|
|           1|928837623|
|           2|731845928|
|           3|  4239079|
|           4|  2625552|
|           5|   750045|
+------------+---------+



#### **fare_amount**

In [ ]:
df = df.where(df.fare_amount > 0)

In [ ]:
df.select("Fare_amount").describe().toPandas()

,summary,Fare_amount
0,count,1747460403
1,mean,12.074943575751059
2,stddev,74.28022857229472
3,min,0.01
4,max,998310.03


In [ ]:
bin_labels_fare = spark.createDataFrame([
    (0, "0-1"),
    (1, "1-2"),
    (2, "2-5"),
    (3, "5-10"),
    (4, "10-20"),
    (5, "20-50"),
    (6, "50-100"),
    (7, "100-200"),
    (8, "200-500"),
    (9, "500-1000"),
    (10, "1000+"),
], ["bins", "range"])

bucketizer_fare = Bucketizer(splits=[0, 1 , 2, 5, 10, 20, 50, 100, 200, 500, 1000, float('Inf')],
                        inputCol='fare_amount',
                        outputCol='bins')

df_bins_fare = bucketizer_fare.setHandleInvalid('keep').transform(df)

df_binned_fare = df_bins_fare.groupBy('bins').count()

result_fare = (df_binned_fare
    .join(bin_labels_fare, "bins")
    .select("range", "count","bins").orderBy("bins")
)

result.show()

+--------+----------+----+
|   range|     count|bins|
+--------+----------+----+
|    0-20|1655162267| 0.0|
|   20-50|  83294409| 1.0|
|  50-100|   8929389| 2.0|
| 100-200|     72393| 3.0|
| 200-500|      1834| 4.0|
|500-1000|       111| 5.0|
+--------+----------+----+



#### **tip_amount**

In [ ]:
bin_labels_tips = spark.createDataFrame([
    (0, "0-1"),
    (1, "1-2"),
    (2, "2-5"),
    (3, "5-10"),
    (4, "10-20"),
    (5, "20-50"),
    (6, "50-100"),
    (7, "100-200"),
    (8, "200-500"),
    (9, "500-1000"),
    (10, "1000+"),
], ["bins", "range"])

bucketizer_tips = Bucketizer(splits=[0, 1,2,5,10,20, 50, 100, 200, 500, 1000, float('Inf')],
                        inputCol='fare_amount',
                        outputCol='bins')

df_bins_tips = bucketizer_tips.setHandleInvalid('keep').transform(df)

df_binned_tips = df_bins_tips.groupBy('bins').count()

result_tips = (df_binned_tips
    .join(bin_labels_tips, "bins")
    .select("range", "count","bins").orderBy("bins")
)

In [ ]:
result_tips.show()

+--------+---------+----+
|   range|    count|bins|
+--------+---------+----+
|     0-1|    44881| 0.0|
|     1-2|     3233| 1.0|
|     2-5|194102117| 2.0|
|    5-10|796952193| 3.0|
|   10-20|537496552| 4.0|
|   20-50|183067289| 5.0|
|  50-100| 34949907| 6.0|
| 100-200|   758279| 7.0|
| 200-500|    83810| 8.0|
|500-1000|     1697| 9.0|
|   1000+|      445|10.0|
+--------+---------+----+



Looking at tips over $1000. Clearly some errors when distance and the corresponding fare are far from the official rate. Likely that nobody got a nearly 400k tip.

In [ ]:
df.where(df.tip_amount > 1000).show()

+---------------+-------------+------------+-----------+----------+--------------------+---------------------+------------+------------+
|passenger_count|Trip_Distance|payment_type|fare_amount|tip_amount|tpep_pickup_datetime|tpep_dropoff_datetime|PULocationID|DOLocationID|
+---------------+-------------+------------+-----------+----------+--------------------+---------------------+------------+------------+
|              1|         5.32|           1|       22.0| 3950588.8| 2015-01-18 19:24:15|  2015-01-18 19:51:55|         249|          37|
|              1|          4.6|           1|     8014.5|   1603.05| 2015-02-26 21:53:48|  2015-02-26 22:15:02|         264|         264|
|              1|          1.2|           1|     8004.5|    1200.8| 2015-09-20 21:50:26|  2015-09-20 21:52:57|         264|         264|
|              1|          5.5|           1|     8000.3|   1600.22| 2017-05-02 12:39:43|  2017-05-02 12:43:41|         263|         236|
|              1|          5.6|          

Zooming in between $500 and $550 in tip. Some trip distances and fare don't add up

In [ ]:
df.where((df.tip_amount > 500) & (df.tip_amount <550)).show()

+---------------+-------------+------------+-----------+----------+--------------------+---------------------+------------+------------+
|passenger_count|Trip_Distance|payment_type|fare_amount|tip_amount|tpep_pickup_datetime|tpep_dropoff_datetime|PULocationID|DOLocationID|
+---------------+-------------+------------+-----------+----------+--------------------+---------------------+------------+------------+
|              1|          6.3|           2|       17.3|     511.5| 2010-02-12 02:26:01|  2010-02-12 02:47:23|         144|         238|
|              2|         18.3|           1|       48.5|    542.41| 2013-08-27 19:19:51|  2013-08-27 19:43:41|         132|         252|
|              1|          4.9|           1|       16.5|    523.42| 2014-08-03 02:08:36|  2014-08-03 02:25:31|         100|           7|
|              1|          7.8|           1|       24.5|    500.11| 2014-11-28 13:13:30|  2014-11-28 13:32:30|         138|         167|
|              1|          3.0|          

Here we can clean by checking if the fare makes sense with the mileage and time
The minimum ride is 3, then 2.18 per fith of a mile.
The time is 0.5/minute if the taxi is not moving.

Exemple 1
calc fare based on mileage (3+ (16.8 - 0.2)*2.18) = 40.2
calc fare based on time 2.5 hours -> 150 min -> 75

102.5 < 40.2 + 75

the total fare is less than 102.5 < 75+63
What is the speed?

75 - 63.12 = 11.88 -> almost 24 minutes at less than 12 mph

We could deduce the minimum speeds and check if they make sense

Exemple 2:

(3 + (4.51 -0.2)*2.18 = 12.37
13 minutes -> 26

6964 > 26 + 12.37
The fare is more than the price per minute + the mileage: it doesn't make sense, this trip data is wrong.

In [ ]:
df_errors = df.where(
    (df.Trip_Distance>50) & \
    # If the fare is more than the rate/mile + rate/min, there is really something wrong)
    (df.fare_amount >= (df.Trip_Distance-0.2 * 2.5)+2.5 + (unix_timestamp(df.tpep_dropoff_datetime) - unix_timestamp(df.tpep_pickup_datetime))/60*2.18) & \
    # Speed = distance / time (in hours)
    (df.Trip_Distance /
     (unix_timestamp(df.tpep_dropoff_datetime) - unix_timestamp(df.tpep_pickup_datetime)) * 3600.0 <= 60.0)
)

## Calculations

In [ ]:

fig, axes = plt.subplots(nrows=1, ncols=1)
fig.set_size_inches(20, 20)

filtered = df.select("Trip_Distance")
# Use the hist function to plot histograms on the Axes
hist(axes, filtered, bins = 20, color=['red'])
axes.set_title('Distance')
axes.legend()

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have the histogram data in a pandas DataFrame
hist_data = df.groupBy("payment_type").count().toPandas()

# Plotting the histogram
#plt.bar(hist_data['payment_type'], hist_data['count'])
#plt.xlabel('payment_type')
#plt.ylabel('Count')
#plt.title('Histogram of payment type')
#plt.show()

In [ ]:
df.agg({'Fare_amount': 'sum'}).show()

In [ ]:
df.agg({'Tip_amount': 'sum'}).show()